In [1]:
!pip install kafka-python
!pip install s3fs

In [1]:
import pandas as pd
from kafka import KafkaConsumer
from time import sleep
from json import loads,dumps
from s3fs import S3FileSystem
import time
import os
from dotenv import load_dotenv

In [53]:
load_dotenv(dotenv_path="../.env")
KAFKA_IP = os.getenv("KAFKA_BROKER_IP")
BUCKET_NAME = os.getenv("S3_BUCKET_NAME")

consumer = KafkaConsumer(
    #'demo_test',
    'bursaMarketShare',
    bootstrap_servers=[KAFKA_IP],
    value_deserializer=lambda x: loads(x.decode('utf-8')),
    auto_offset_reset='earliest',
    consumer_timeout_ms=5000  # stop after 5 seconds of inactivity
)
all_data = [msg.value for msg in consumer] 
#kafka lazy af , If no new messages are coming in, they sit idle forever which cause no message detected hence collect msg when it starts is better

'''
optional ->by default, AWS Glue Crawlers automatically lowercase column names when inferring schema from JSON files,which create
rows in Athena but the columns are blank, as files on s3 use a different casing to name the columns
'''
#def lowercase_keys(d):
    #return {k.lower(): v for k, v in d.items()}

#lowered_data = [lowercase_keys(row) for row in all_data]


'\noptional ->by default, AWS Glue Crawlers automatically lowercase column names when inferring schema from JSON files,which create\nrows in Athena but the columns are blank, as files on s3 use a different casing to name the columns\n'

In [4]:

#print (message .value)
print (all_data[:5])


[{'No': 1, 'CASHTAG': '$SEVE', 'NAME': "7-ELEVEN M'SIA", 'PRICE': 1.99, 'CLOSE': 1.99, 'CHG': 0.0, '%CHG': '0.000%', 'VOLUME': 0, 'HIGH': 1.99, 'LOW': 1.99, 'BUY (QTY)': '1.980 (500)', 'SELL (QTY)': '2.000 (116900)'}, {'No': 3, 'CASHTAG': '$ARNK', 'NAME': 'A-RANK BHD', 'PRICE': 0.435, 'CLOSE': 0.44, 'CHG': 0.0, '%CHG': '0.000%', 'VOLUME': 0, 'HIGH': 0.45, 'LOW': 0.44, 'BUY (QTY)': '0.425 (4500)', 'SELL (QTY)': '0.480 (7000)'}, {'No': 5, 'CASHTAG': '$ABLE', 'NAME': 'ABLE GLOBAL', 'PRICE': 1.52, 'CLOSE': 1.53, 'CHG': -0.01, '%CHG': '-0.650%', 'VOLUME': 94000, 'HIGH': 1.52, 'LOW': 1.51, 'BUY (QTY)': '1.510 (59600)', 'SELL (QTY)': '1.520 (19000)'}, {'No': 8, 'CASHTAG': '$ACME', 'NAME': 'ACME HOLDGS BHD', 'PRICE': 0.125, 'CLOSE': 0.13, 'CHG': -0.005, '%CHG': '-3.850%', 'VOLUME': 11000, 'HIGH': 0.13, 'LOW': 0.13, 'BUY (QTY)': '0.125 (20000)', 'SELL (QTY)': '0.130 (500)'}, {'No': 10, 'CASHTAG': '$REN3', 'NAME': '3REN BERHAD', 'PRICE': 0.255, 'CLOSE': 0.25, 'CHG': 0.01, '%CHG': '4.080%', 'VOLU

In [33]:
s3 = S3FileSystem()
#Per-message write
for count, i in enumerate(all_data):
    with s3.open(f"s3://{BUCKET_NAME}/bursamarketshare{count}.json", 'w') as file:
        json.dump(i, file)

In [55]:
s3 = S3FileSystem()
timestamp = time.strftime("%Y%m%d_%H%M%S") 
"""
crawler has issues with with mixed file formats so choose one dont be greedy
"""
#json
file_name = f"s3://{BUCKET_NAME}/bursamarketshare_{timestamp}.json"
#csv option
#file_name = f"s3://demotestzakuan/bursamarketshare_{timestamp}.csv"
df = pd.DataFrame(all_data)
with s3.open(file_name, 'w') as file:
    #json.dump(all_data[:5], file,indent=2)
    #json.dump(all_data, file,indent=2) 
    #athena becomes dump cuz above line producing JSON with a top-level array, which Athena cannot query directly
    
    # Write NDJSON (newline-delimited JSON) file
    df.to_json(file, orient='records', lines=True)

    #csv option
    #df.to_csv(file, index=False)
#Batch process all messages into one file
